# Examining Point Clouds

This notebook walks through the first default phase of a Spyral pipeline, `PointcloudPhase`. For documentation on the phases, follow this [link](https://attpc.github.io/Spyral/user_guide/phases/about/) to the documentation.

### Imports

First we import all of the necessary modules

In [ ]:
from spyral.core.point_cloud import point_cloud_from_get, calibrate_point_cloud_z
from spyral.core.run_stacks import form_run_string
from spyral.trace.get_event import GetEvent, GET_DATA_TRACE_START, GET_DATA_TRACE_STOP
from spyral.trace.trace_reader import create_reader
from spyral.correction import create_electron_corrector
from spyral.core.pad_map import PadMap

from spyral import PadParameters, GetParameters, FribParameters, DetectorParameters, DEFAULT_MAP, PointcloudPhase 

import numpy.random as random
import numpy as np
from pathlib import Path
import plotly.graph_objects as go
from plotly.subplots import make_subplots

def find_trace_from_padid(event: GetEvent, pad_id: int) -> int:
    for idx, trace in enumerate(event.traces):
        if trace.hw_id.pad_id == pad_id:
            return idx
    return -1

### Configuration

Define your Spyral configuration below. If you aren't sure what some of these values mean, they are all documented at the [Spyral documentation](https://attpc.github.io/Spyral/user_guide/config/about/)

In [ ]:
# Paths to your data
trace_path = Path("/path/to/raw/attpc/traces/")
workspace_path = Path("/path/to/your/workspace/")

# Pad mapping. We use defaults here
pad_params = PadParameters(
    pad_geometry_path=DEFAULT_MAP,
    pad_time_path=DEFAULT_MAP,
    pad_electronics_path=DEFAULT_MAP,
    pad_scale_path=DEFAULT_MAP,
)

# AT-TPC GET trace analysis
get_params = GetParameters(
    baseline_window_scale=20.0,
    peak_separation=50.0,
    peak_prominence=20.0,
    peak_max_width=50.0,
    peak_threshold=40.0,
)

# AT-TPC FRIBDAQ trace analysis
frib_params = FribParameters(
    baseline_window_scale=100.0,
    peak_separation=50.0,
    peak_prominence=20.0,
    peak_max_width=500.0,
    peak_threshold=100.0,
    ic_delay_time_bucket=1100,
    ic_multiplicity=1,
)

# Detector properties
det_params = DetectorParameters(
    magnetic_field=2.85,
    electric_field=45000.0,
    detector_length=1000.0,
    beam_region_radius=25.0,
    micromegas_time_bucket=10.0,
    window_time_bucket=560.0,
    get_frequency=6.25,
    garfield_file_path=Path("/path/to/some/garfield.txt"),
    do_garfield_correction=False,
)

### Setup the Data 

Now that our configuration is loaded, we can start reading and analyzing some data. Step one is to access the raw trace datafile. This means that you need to pick a run to analyze; we store the run number in a variable for later reference. To analyze a different run simply change the run number. 

Traces come in some different formats from tools like [attpc_merger](https://github.com/ATTPC/attpc_merger) or [harmonizer](https://github.com/ATTPC/harmonizer). We use the TraceReader protocol to handle which format we're looking at. The `create_reader` function choses the appropriate implementation.


In [ ]:
run_number = 16 # pick a run
trace_file_path = trace_path / f"{form_run_string(run_number)}.h5"
trace_reader = create_reader(trace_file_path, run_number)
if trace_reader is None:
    raise Exception("Invalid Reader! Make sure the traces exist!")

Now we'll setup an iterator so we can walk through the events in the file. This will allow us to walk through events in order. Only run this code block **ONCE**. If you run it again you'll start the iterator over and just keep looking at the first event! We'll also load some assets here that we'll use later in the analysis.

In [ ]:
# Ask the trace file for the range of events, and make an iterator
event_iterator = iter(trace_reader.event_range())
rng = random.default_rng()
phase = PointcloudPhase(get_params, frib_params, det_params, pad_params)
phase.create_assets(workspace_path)
correction_path = phase.electron_correction_path
pad_map = PadMap(pad_params)


### Analyzing

Everything below this section can be run repeatedly to walk through the data sequentially.

We retrieve the next event in the file (or the current hardcoded event if you want) and do some signal analysis

In [ ]:
event_number = next(event_iterator)
# You can also hardcode an event number here!
# event_number = 1
print(f"Analzying event: {event_number}")
event = trace_reader.read_event(event_number, get_params, frib_params, rng) # The signal-analyzed event
raw_event_data = trace_reader.read_raw_get_event(event_number) # The raw GET data for comparision

First, we'll look at the signal analysis on a random trace within the event. 

In [ ]:
trace_number = random.randint(0, len(raw_event_data))
# trace_number = find_trace_from_padid(event, 397)
raw_trace_data = raw_event_data[trace_number]
time_bucket_range = np.arange(start=0, stop=512)

fig = go.Figure()
fig.add_trace(
    go.Scatter(x=time_bucket_range, y=raw_trace_data[GET_DATA_TRACE_START:GET_DATA_TRACE_STOP], mode="lines", name=f"Raw Trace {trace_number}")
)
fig.add_trace(
    go.Scatter(x=time_bucket_range, y=event.traces[trace_number].trace, mode="lines",name=f"Baseline Corrected Trace {trace_number}")
)
print(f"Trace Number: {trace_number}")
print(f"Trace Hardware: {event.traces[trace_number].hw_id}")
peak_amps = []
peak_cents = []
peak_left = []
peak_left_amps = []
peak_right = []
peak_right_amps = []
for peak in event.traces[trace_number].get_peaks():
    peak_amps.append(peak.amplitude)
    peak_cents.append(np.floor(peak.centroid))
    peak_left.append(peak.positive_inflection)
    peak_right.append(peak.negative_inflection)
    peak_left_amps.append(event.traces[trace_number].trace[int(peak.positive_inflection)])
    peak_right_amps.append(event.traces[trace_number].trace[int(peak.negative_inflection)])
print(f"Peak centroids: {peak_cents}")
fig.add_trace(
    go.Scatter(x=peak_cents, y=peak_amps, mode="markers", name="Peaks")
)
fig.add_trace(
    go.Scatter(x=peak_left, y=peak_left_amps, mode="markers", name="Peak Left Edges")
)
fig.add_trace(
    go.Scatter(x=peak_right, y=peak_right_amps, mode="markers", name="Peak Right Edges")
)
fig.update_legends()
fig.update_layout(
    xaxis_title="Time Bucket",
    yaxis_title="Amplitude",
    showlegend=True
)
fig.show()

Above you should see the plot of the raw trace as well as the baseline corrected trace. The baseline corrected trace is computed using a low-pass filter. The peaks are labeled with their centroids and left and right edges. To look at different traces, you can run the above cell over and over again; it will select a random trace in the event each time.

Now that we have our signals, we will use the pad information (x, y) and the signal time (time bucket) to create a 3-D image of the whole event, called a PointCloud.

In [ ]:
cloud = point_cloud_from_get(event.get, pad_map)
hover_text = [f"Pad ID: {int(point[5])}" for point in cloud.data] # We'll use this later

fig = make_subplots(2, 1, row_heights=[0.66, 0.33], specs=[[{"type": "xy"}], [{"type": "scene"}]])
fig.add_trace(
    go.Scatter3d(
        x=cloud.data[:, 2], 
        y=cloud.data[:, 0], 
        z=cloud.data[:, 1], 
        mode="markers",
        text = hover_text,
        hovertemplate="X: %{y:.2f}<br>Y: %{z:.2f}<br>Z: %{x:.2f}<br>%{text}",
        marker= {
            "size": 3, 
            "color": cloud.data[:, 3], 
            "showscale": True
            }, 
        name="Point Cloud"
    ),
    row=2,
    col=1
)
fig.add_trace(
    go.Scatter(
        x=cloud.data[:, 0], 
        y=cloud.data[:, 1], 
        mode="markers",
        text = hover_text,
        hovertemplate="X: %{x:.2f}<br>Y: %{y:.2f}<br>%{text}",
        marker= {
            "color": cloud.data[:, 3], 
            "showscale": True
        }, 
        name="XY Projection"),
    row=1,
    col=1
)
fig.update_layout(
    xaxis_title = "X (mm)",
    yaxis_title = "Y (mm)",
    xaxis_range=[-300.0, 300.0],
    yaxis_range=[-300.0, 300.0],
    scene = {
        "xaxis_title": "Z (Time Buckets)",
        "yaxis_title": "X (mm)",
        "zaxis_title": "Y (mm)",
        "aspectratio": {
            "x": 3.3,
            "y": 1.0,
            "z": 1.0
        },
        "xaxis_range": [0.0, 512.0],
        "yaxis_range": [-300.0, 300.0],
        "zaxis_range": [-300.0, 300.0],
    },
    width = 1000,
    height = 1500,
    showlegend=False
)
fig.show()

Above, you should see your point cloud plotted in 3D as well as the X-Y plane (pad plane) projection. The marker color indicates the charge ampltiude of the peak used to make the point in the point cloud. If you hover over one of the points in either plot, you'll see a label which shows the coordinate position as well as the trace and peak number which produced the point. This can be used to pick specific traces to examine

The z-axis is still in Time Buckets. We would like to convert this time axis into a position. To do this we use the reference time of the window and micromegas mesh (i.e. the position of the ends of the detector within the trigger). These values have to be estimated from the data. Typically this is handled by looking for window events (events where the beam reacted with the window), because they typically span the entire volume of the detector. Once you've set these values in your config, run the cell below to re-plot the point cloud with calibrated z-position.

When calibrating we also apply an electric field correction from a Garfield++ simulation of the AT-TPC electric field. This allows us to correct for field non-uniformities, particularly near the edges of the AT-TPC.

In [ ]:
# Load the correction if requested
corrector = None
if correction_path.exists():
    corrector = create_electron_corrector(correction_path)

calibrate_point_cloud_z(cloud, det_params, efield_correction=corrector)

fig = go.Figure()
fig.add_trace(
    go.Scatter3d(
        x=cloud.data[:, 2], 
        y=cloud.data[:, 0], 
        z=cloud.data[:, 1], 
        mode="markers", 
        marker= {
            "size": 3, 
            "color": cloud.data[:, 4], 
            "showscale": True
        }, 
        name="Point Cloud"
    )
)
fig.update_layout(
    scene = {
        "xaxis_range": [0.0, 1000.0],
        "yaxis_range": [-300.0, 300.0],
        "zaxis_range": [-300.0, 300.0],
        "xaxis_title": "Z (mm)",
        "yaxis_title": "X (mm)",
        "zaxis_title": "Y (mm)",
        "aspectratio": {
            "x": 3.3,
            "y": 1.0,
            "z": 1.0
        }
    },
    height=750,
)
fig.show()

There is still more you can look at however. You can even plot some interesting physics! Below is an example intended to try and plot a Bragg curve from the point cloud.

In [ ]:
# Plot r-Charge projection
fig = go.Figure()
fig.add_trace(
    go.Scatter(x=np.linalg.norm(cloud.data[:, :3], axis=1), y=cloud.data[:, 4], mode="markers", marker={"size": 5})
)
fig.update_layout(
    xaxis_title="Position (mm)",
    yaxis_title="Integral"
)
fig.show()

You can re-run the code cells in this Analyzing section to walk through the data sequentially, or select a specific event to look at.

### Conclusion

That is a basic analysis of the traces and the point cloud data! With well tuned parameters, you're now ready to run the phase 1 analysis. Follow the instructions in the README to do this. Once thats done, you can move on to the next stage, generating and identifying clusters within the point clouds.